In [1]:
import pandas as pd
import numpy as np
import regex as re
from nltk.tokenize import word_tokenize
from google.colab import drive

!pip install transformers
!pip install sentencepiece
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AdamW
from transformers import RobertaModel, RobertaTokenizer
from scipy.spatial.distance import cosine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader,RandomSampler, SequentialSampler
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.nn.utils.rnn import pad_sequence
from transformers import XLMRobertaConfig, XLMRobertaModel, XLMRobertaTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import os
import seaborn as sns
import matplotlib.pyplot as plt


from google.colab import drive
drive.mount('/content/drive')
LIBRARY_PATH = '/content/drive/MyDrive/My Project/'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.8 MB/s eta 0:00:00
Mounted at /content/drive


# Data Preprocessing

In [2]:
original_train_path = 'Train Data/train_df.csv'
original_train = pd.read_csv(LIBRARY_PATH + original_train_path)

In [3]:
# Check the size of the original dataframe.
original_size = original_train.shape
original_num_rows = original_size[0]
original_num_columns = original_size[1]

print(f"Number of rows: {original_num_rows}")
print(f"Number of columns: {original_num_columns}")
original_train.head(2)

Number of rows: 4955
Number of columns: 16


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,text1,text2
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,1.0,4.0,4.000000,1.666667,2.000000,"MARTINSBURG, W.Va. — A suspected drunken drive...","PORT-AU-PRINCE, Haiti — Haitian President Jove..."
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,1.0,4.0,3.666667,1.666667,1.333333,(NEWS ROOM GUYANA) — Three persons are current...,Sign In The Star Edition Change Location Sign ...


In [4]:
# Delete columns that contain NaN sentences.
processed_data = original_train[original_train['text1'].notna()]
train_data = processed_data[processed_data['text2'].notna()]

print("After removing NA text columns, we lose {0} rows.".format(original_train.shape[0] - train_data.shape[0]))

After removing NA text columns, we lose 113 rows.


In [5]:
# Truncation of dataset.
train_data = train_data.head(2000)

# Roberta Implementation

In [6]:
# Load pre-trained model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# Define the encode_article and calculate_similarity functions.
def encode_article(article):
    input_ids = tokenizer.encode(article, return_tensors='pt', max_length=512, truncation=True)
    embeddings = model(input_ids)[0].mean(1).detach().numpy().flatten()
    return embeddings

def calculate_similarity(article1, article2):
    embeddings1 = encode_article(article1)
    embeddings2 = encode_article(article2)
    similarity = 1 - cosine(embeddings1, embeddings2)
    return similarity

# Calculate similarity for each row in the DataFrame.
similarities = []
for index, row in train_data.iterrows():
    similarity = calculate_similarity(row['text1'], row['text2'])
    similarities.append(similarity)

# Add the similarities to the DataFrame as a new column.
train_data['similarity'] = similarities

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Normalize the similarity scores to [1, 4].
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 3))
train_data['normalized_similarity'] = 4 - min_max_scaler.fit_transform(train_data[['similarity']])

In [8]:
# Calculate the weighted gold similarity.
scores_sum = ['Geography', 'Entities', 'Time', 'Narrative', 'Overall', 'Style', 'Tone']
train_data['weighted_gold_similarity'] = train_data[scores_sum].sum(axis=1) / 7

In [10]:
# Calculate the Pearson correlation coefficient.
pearson = train_data['normalized_similarity'].corr(train_data['weighted_gold_similarity'])
print("Pearson correlation coefficient:", pearson)

Pearson correlation coefficient: 0.06957549803199095
